# CPC MJO Replication

In [5]:
import numpy as np
import netCDF4 as nc
import os, sys, time, re
import math
import datetime
from datetime import date, timedelta
from scipy.interpolate import griddata

### Define the Data Array

In [6]:
def date_range(start, end):
    days = []
    delta = end - start
    for i in range(delta.days+1):
        days.append((start+timedelta(days=i)).strftime("%Y-%m-%d"))
    return days

start_date = datetime.date(1950, 1, 1)
end_date = datetime.date(2023, 2, 28)
datesd = date_range(start_date, end_date)

num_days = len(datesd)

In [14]:
print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")
print(f"{datesd[:2]} to {datesd[-2:]}")
print(f"Number of days: {num_days}")

Start Date: 1950-01-01
End Date: 2023-02-28
['1950-01-01', '1950-01-02'] to ['2023-02-27', '2023-02-28']
Number of days: 26722


### Define Target and Data Dimensions

Define dimensions, using only the tropical latitudes (-15 to 15 degrees)

In [15]:
min_lat = -15
max_lat = 15

In [18]:
target_lat = np.arange(-15.00, 15.0+0.5, 2.5)
target_lon = np.arange(1.25, 360, 2.5)
target_x, target_y = np.meshgrid(target_lon, target_lat)

In [32]:
# Full Range
clon = np.arange(0,360,1.0)
clat = np.arange(-90,90+0.5,1.0)

In [33]:
# Range of values between the max and min latitude values
cy = np.where(np.logical_and(clat>=min_lat, clat<=max_lat))[0]
print(cy)

[ 75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104 105]


In [34]:
# Filter clat based on range of min/max latitude values
print(f"Original: {clat}")
clat = clat[cy]
print(f"Filtered: {clat}")

Original: [-90. -89. -88. -87. -86. -85. -84. -83. -82. -81. -80. -79. -78. -77.
 -76. -75. -74. -73. -72. -71. -70. -69. -68. -67. -66. -65. -64. -63.
 -62. -61. -60. -59. -58. -57. -56. -55. -54. -53. -52. -51. -50. -49.
 -48. -47. -46. -45. -44. -43. -42. -41. -40. -39. -38. -37. -36. -35.
 -34. -33. -32. -31. -30. -29. -28. -27. -26. -25. -24. -23. -22. -21.
 -20. -19. -18. -17. -16. -15. -14. -13. -12. -11. -10.  -9.  -8.  -7.
  -6.  -5.  -4.  -3.  -2.  -1.   0.   1.   2.   3.   4.   5.   6.   7.
   8.   9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.  21.
  22.  23.  24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.  35.
  36.  37.  38.  39.  40.  41.  42.  43.  44.  45.  46.  47.  48.  49.
  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.  60.  61.  62.  63.
  64.  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.  75.  76.  77.
  78.  79.  80.  81.  82.  83.  84.  85.  86.  87.  88.  89.  90.]
Filtered: [-15. -14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4. 

In [36]:
# Input is the full range of longtiude, but filtered to only tropical lattitudes
input_x, input_y = np.meshgrid(clon, clat)

### Collect Staged Data in .nc Format

In [38]:
# Set up zero meshes for MJO formats u850, u200, and olr
u850 = np.zeros([num_days,len(clat),len(clon)],dtype='f')
u200 = np.zeros([num_days,len(clat),len(clon)],dtype='f')
olr = np.zeros([num_days,len(clat),len(clon)],dtype='f')

### Reading in U850 NetCDF4 Data

In [39]:
infile = nc.Dataset('input_daily_u850_anom_data', mode='r')

FileNotFoundError: [Errno 2] No such file or directory: 'input_daily_u850_anom_data'